In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
import os
import numpy as np
import zipfile

/Users/dariaageikina/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [14]:
# Set up the working directory
main_path = os.getenv("DATA_PATH")
os.chdir(main_path + "/carb_offsets/data")

In [4]:
#load parcels
parcels0 = gpd.read_file("build_forest_parcels/output/forest_parcels1km.gpkg")
parcels = parcels0.to_crs('EPSG:5070')
parcels['parcel_id'] = parcels.index + 1
parcels['parcel_area'] = parcels.geometry.area

In [6]:
#merge in wildfire hazard potential
whp_path = 'build_fire_risk/input/raw/new_wildfire_hazard_potential/2014/RDS-2015-0047/Data/whp_2014_continuous/whp2014_cnt'
stats = zonal_stats(parcels, whp_path, stats='mean', nodata=-2147483647)
parcels['whp_2014'] = [stat['mean'] for stat in stats]
parcels = parcels.dropna(subset=['whp_2014'])
parcels.reset_index(drop=True, inplace=True)

In [7]:
#merge in conservation easement data
CEs = gpd.read_file("build_conservation_easement/input/raw/NCED_08282020_shp")
CEs = CEs.to_crs(parcels.crs)
CEs = CEs[CEs['owntype']!='FED']
CEs = CEs[['unique_id','geometry']]

CEs.rename(columns={'unique_id': 'ce_id'}, inplace=True)
intersected = gpd.sjoin(parcels, CEs, how='left', predicate = 'intersects')

intersected = intersected.groupby('parcel_id').agg({
    'ce_id': lambda x: ', '.join(x.astype(str))
}).reset_index()

intersected['ce'] = 0
intersected.loc[intersected['ce_id']!="nan", 'ce'] = 1
intersected[intersected['ce']==1]
intersected = intersected[['parcel_id','ce']]

parcels = parcels.merge(intersected, on='parcel_id', how='left')

In [8]:
#merge in canopy cover
canopy_path14 = 'build_canopy_cover/input/raw/NLCD_2014/nlcd_tcc_conus_2014_v2021-4.tif'

stats = zonal_stats(parcels, canopy_path14, stats='mean', nodata=None)
parcels['canopy_2014'] = [stat['mean'] for stat in stats]
parcels = parcels.dropna(subset=['canopy_2014'])
parcels.reset_index(drop=True, inplace=True)

/Users/dariaageikina/anaconda3/lib/python3.11/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [9]:
#merge in eco regions
eco_region_gpds = []

for region_num in range(1,11):
        eco_region = gpd.read_file('build_eco_regions/input/raw/level4/reg'+str(region_num)+'_eco_l4')
        eco_region = eco_region.to_crs('EPSG:5070')
        
        eco_region_gpds.append(eco_region)
 
eco_regions = gpd.pd.concat(eco_region_gpds)

eco_regions.drop(columns=['OBJECTID','L4_KEY','L3_KEY','L2_KEY','L1_KEY','Shape_Leng','Shape_Area'], inplace=True)

intersected = gpd.overlay(parcels, eco_regions, how='intersection')
intersected['intersect_area'] = intersected.geometry.area
intersected = intersected[['parcel_id','US_L4CODE','US_L4NAME','US_L3CODE',
                           'US_L3NAME','NA_L3CODE','NA_L3NAME','NA_L2CODE',
                           'NA_L2NAME','NA_L1CODE','NA_L1NAME','STATE_NAME',
                           'EPA_REGION','intersect_area']]

aggregated_df = intersected.loc[
    intersected.groupby(['parcel_id'])['intersect_area'].idxmax()
]
aggregated_df = aggregated_df.reset_index(drop=True)
aggregated_df.drop(columns=['intersect_area'],inplace=True)

parcels = parcels.merge(aggregated_df, on='parcel_id', how='left')

In [12]:
#merge in projects
projects = gpd.read_file("build_ARB/input/built/all_projects.gpkg")
projects = projects[projects.geometry.type.isin(['Polygon', 'MultiPolygon'])]
projects = projects.to_crs(parcels.crs)

intersected = gpd.overlay(parcels, projects, how='intersection')

intersected_union = intersected.dissolve(by="parcel_id", as_index=False)
intersected_union['intersect_area'] = intersected.geometry.area
intersected_union.drop(columns=["project","type"],inplace=True)

intersected = intersected[['parcel_id','project','type']]

intersected = intersected.groupby('parcel_id').agg({
    'project': lambda x: ', '.join(x.astype(str)),
    'type': lambda x: ', '.join(x.astype(str)),
}).reset_index()
intersected = intersected.merge(intersected_union[['parcel_id','intersect_area']])

parcels = parcels.merge(intersected, on='parcel_id', how='left')
parcels['intersect_area'] = parcels['intersect_area'].fillna(0)
parcels['project_share'] = parcels['intersect_area'] / parcels['parcel_area']
parcels.drop(columns=['intersect_area'], inplace=True)

/var/folders/bm/prztv6jj77l86gjc3j8qp4pr0000gn/T/ipykernel_5781/3700971061.py:6: UserWarning: `keep_geom_type=True` in overlay resulted in 5 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersected = gpd.overlay(parcels, projects, how='intersection')


In [15]:
#merge in census tracts
census_tracts = gpd.read_file(main_path+"/wfs_media/data/build_wildfire_reports/input/raw/census_tracts/nhgis0001_shape/2014")
census_tracts = census_tracts.to_crs(parcels.crs)

intersected = gpd.overlay(parcels, census_tracts[['GEOID','geometry']], how='intersection')
intersected['intersect_area'] = intersected.geometry.area
intersected = intersected[['parcel_id','GEOID','intersect_area']]

aggregated_df = intersected.loc[
    intersected.groupby(['parcel_id'])['intersect_area'].idxmax()
]
aggregated_df = aggregated_df.reset_index(drop=True)
aggregated_df.drop(columns=['intersect_area'],inplace=True)

parcels = parcels.merge(aggregated_df, on='parcel_id', how='left')

In [16]:
#merge in zipcodes
zipcodes = gpd.read_file(main_path+"/wfs_media/data/build_wildfire_reports/input/raw/zipcodes/USA_ZIP_Code_Boundaries.gpkg")
zipcodes = zipcodes[['ZIP_CODE','geometry']]

intersected = gpd.overlay(parcels, zipcodes, how='intersection')
intersected['intersect_area'] = intersected.geometry.area
intersected = intersected[['parcel_id','ZIP_CODE','intersect_area']]

aggregated_df = intersected.loc[
    intersected.groupby(['parcel_id'])['intersect_area'].idxmax()
]
aggregated_df = aggregated_df.reset_index(drop=True)
aggregated_df.drop(columns=['intersect_area'],inplace=True)

parcels = parcels.merge(aggregated_df, on='parcel_id', how='left')

In [17]:
#merge in counties
counties = gpd.read_file(main_path+"/wfs_media/data/build_wildfire_reports/input/raw/county_boundaries/gz_2010_us_050_00_20m")
counties = counties.to_crs(parcels.crs)
counties.loc[:,'fips'] = (counties['STATE']+counties['COUNTY']).astype(int)
counties

intersected = gpd.overlay(parcels, counties, how='intersection')
intersected['intersect_area'] = intersected.geometry.area
intersected = intersected[['parcel_id','fips','intersect_area']]
intersected

aggregated_df = intersected.loc[
    intersected.groupby(['parcel_id'])['intersect_area'].idxmax()
]
aggregated_df = aggregated_df.reset_index(drop=True)
aggregated_df.drop(columns=['intersect_area'],inplace=True)

parcels = parcels.merge(aggregated_df, on='parcel_id', how='left')

In [18]:
#merge in housing prices
housing = pd.read_csv("build_housing/input/raw/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
housing['housing_value2014'] = housing.filter(regex='^2014').mean(axis=1, skipna=True)
housing = housing[['RegionName','housing_value2014']]
housing.rename(columns={"RegionName":"ZIP_CODE"}, inplace=True)
housing['ZIP_CODE'] = housing['ZIP_CODE'].astype(str)

parcels = parcels.merge(housing,how="left")

In [19]:
#merge in census data
census = pd.read_csv('build_socioeconomic/input/raw/ACSDP5Y2014/ACSDP5Y2014.DP03-Data.csv')
census2 = pd.read_csv('build_socioeconomic/input/raw/ACSST5Y2014/ACSST5Y2014.S2301-Data.csv')

census = census.merge(census2)

census = census[['GEO_ID','DP03_0030E','DP03_0030PE','DP03_0033E','DP03_0033PE','DP03_0051E',
                'S2301_C01_001E','S2301_C04_001E','S2301_C01_023E','S2301_C01_026E',
                   'S2301_C01_029E','S2301_C01_019E']]

census = census.iloc[1:].reset_index(drop=True)

census.replace("-", np.nan, inplace=True)
census.loc[:, census.columns != 'GEO_ID'] = census.loc[:, census.columns != 'GEO_ID'].astype(float)

for col in census.columns:
    if col!="GEO_ID":
        census[col] = pd.to_numeric(census[col], errors='coerce')

census['GEOID'] = census['GEO_ID'].str[9:]
census['poverty_rate'] = np.where((census['S2301_C01_001E'] != 0) & (~census['S2301_C01_001E'].isna()),
                                  census['S2301_C01_023E'] / census['S2301_C01_001E'],
                                  np.nan)
census['nohighschool_rate'] = np.where((census['S2301_C01_019E'] != 0) & (~census['S2301_C01_019E'].isna()),
                                  census['S2301_C01_026E'] / census['S2301_C01_019E'],
                                  np.nan)
census['collegegrads_rate'] = np.where((census['S2301_C01_019E'] != 0) & (~census['S2301_C01_019E'].isna()),
                                  census['S2301_C01_029E'] / census['S2301_C01_019E'],
                                  np.nan)

census.drop(columns=['GEO_ID','S2301_C01_023E','S2301_C01_026E','S2301_C01_029E'],inplace=True)
census.rename(columns={"DP03_0030E":"jobs1","DP03_0030PE":"jobs1_p","DP03_0033E":"jobs2",
                      "DP03_0033PE":"jobs2_p","DP03_0051E":"income","S2301_C01_001E":"pop16plus",
                       "S2301_C04_001E":"unemp_rate","S2301_C01_019E":"pop20_64"}, inplace=True)

parcels = parcels.merge(census,how="left")

/var/folders/bm/prztv6jj77l86gjc3j8qp4pr0000gn/T/ipykernel_5781/2946491850.py:2: DtypeWarning: Columns (2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,38,39,40,42,43,46,47,50,51,54,55,56,58,59,62,63,64,66,67,70,71,72,74,75,78,79,82,83,86,87,90,91,94,95,102,103,104,106,107,110,111,114,115,118,119,122,123,126,127,128,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,184,186,187,190,191,194,195,198,199,202,203,204,206,207,210,211,214,215,218,219,222,223,226,227,230,231,234,235,238,239,242,243,254,255,262,263,270,271,278,279,286,287,294,295,298,299,300,302,303,306,307,310,311,314,315,318,319,322,323,326,327,330,331,334,335,338,339,354,355,356,378,379,380,382,383,386,387,390,391,394,395,398,399,400,402,403,406,407,408,410,411,412,414,415,416,418,419,422,423,426,427,430,431,434,435,436,438,439,442,443,446,447,450,451,454,455,456,458,459,462,463,466,467,470,471) have mixed types. Specify dtype op

In [ ]:
#export all data
parcels.to_file('merge_data/output/merged_carb.gpkg', layer='alldata', driver='GPKG')